# Create a notebook to perform RK integration for multiple couple variables

In [51]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# Define our coupled derivatives to integrate

In [52]:
def dydx(x,y):
    #set the derivatives
    
    y_derivs = np.zeros(2)
    
    y_derivs[0] = y[1] 
    
    y_derivs[1] = -1*y[0]
    
    return y_derivs

# Define Cash-Karp Method

In [61]:
def cash_karp_core(dydx,xi,yi,nv,h):
    
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    k5 = np.zeros(nv)
    k6 = np.zeros(nv)
    
    #define x at 1/2 step
    x_ipoh = xi + 0.5*h
    
    #define x at one step
    x_ipo = xi + h 
    
    #declare a temp y array
    y_temp = np.zeros(nv)
    
    #get k1 values
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:]
    
    #get k2 values
    y_temp[:] = yi[:] + (1/5)*k1
    y_derivs = dydx(x_ipo + (1/5)*h, y_temp)
    k2[:] = h*y_derivs[:]
    
    #get k3 values
    y_temp[:] = yi[:] + (3/40)*k1 + (9/40)*k2
    y_derivs = dydx(x_ipo + (3/10)*h, y_temp)
    k3[:] = h*y_derivs[:]
    
    #get k4 values
    y_temp[:] = yi[:] + (3/10)*k1 + (-9/10)*k2 + (6/5)*k3
    y_derivs = dydx(x_ipo + (3/5)*h, y_temp)
    k4[:] = h*y_derivs[:]
    
    #get k5 values
    y_temp[:] = yi[:] + (-11/54)*k1 + (5/2)*k2 + (-70/27)*k3 +(35/27)*k4
    y_derivs = dydx(x_ipo + h, y_temp)
    k4[:] = h*y_derivs[:]
    
    #get k6 values
    y_temp[:] = yi[:] + (11631/55296)*k1 + (175/512)*k2 + (575/13824)*k3 +(44275/110592)*k4 + (253/4096)*k5
    y_derivs = dydx(x_ipo + (7/8)*h, y_temp)
    k4[:] = h*y_derivs[:]
    
    #compute yi+1 and yi+1*
    yipo = yi +(37/378)*k1 + 0*k2 + (250/621)*k3 + (125/594)*k4 + 0*k5 + (512/1771)*k6
    yipos = yi +(2825/27648)*k1 + 0*k2 + (18575/48384)*k3 + (13525/55296)*k4 + (277/14336)*k5 + (1/4)*k6
    
    return yipo, yipos
    
    

# Define an adaptive step size driver

In [58]:
def cash_karp_ad(dydx,x_i,y_i,nv,h,tol):
    
    #define safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #set a maaximum number of iterations
    imax = 10000
    
    #set an iteration variable
    i = 0
    
    # create an error
    Delta = np.full(nv,2*tol) #this creates an array of nv
    
    #remember the step
    h_step = h
    
    #adjust step
    while(Delta.max()/tol > 1.0):
        
        #estimate our error by taking one step of size h
        yipo, yipos = cash_karp_core(dydx,x_i,y_i,nv,h_step)
        
        #compute an error
        Delta = np.fabs(yipo - yipos)
        
        #if the error is too large, take a smaller step
        if (Delta.max()/tol > 1.0):
            
            #our error is too large, take a smaller step
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
            
        if(i>=imax):
            print("Too many iterations in cash_karp_ad()")
            raise StopIteration("Ending after i = ",i)
            
        #iterate
        i+=1
        
    #next time, try to take a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-.9), h_step*H_NEW_FAC)
    
    #return the answer, a new step and the step we actually took
    return yipo, h_new, h_step

# Define a wrapper function

In [67]:
def cash_wrap(dydx,a,b,y_a,tol):
    
    #dfdx is the derivative wrt x
    #a is the lower bound
    #b is the yupper bound
    #y_a are the boundary condition
    #tol is the tolerance for integrating y
    
    #define our starting step
    xi = a
    yi = y_a.copy()
    
    #an initial step size == make very small
    h = 1.0e-4 *(b-a)
    
    #set a maximum number of iterations
    imax = 10000
    
    #set an iteration variable
    i = 0
    
    #set the. number of coupled ODEs to the size of y_a
    nv = len(y_a)
    
    #set the initial conditions
    #these are both array: the np.full function makes the array
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    
    #set a flag
    #what does setting a flag do?
    flag = 1
    
    #loop untul we reach the right side
    while(flag):
        
        #calculate y_i+1
        yi_new, h_new, h_step = cash_karp_ad(dydx,xi,yi,nv,h,tol)
        
        #update ethe step
        h = h_new
        
        #prevent an overshoot
        if(xi+h_step>b):
            
            #take a smaller step
            h = b-xi
            
            #recalculate y_i+1
            yi_new, h_new, h_step = cash_karp_ad(dydx,xi,yi,nv,h,tol)
            
            #break
            flag = 0
        #update calues
        xi += h_step
        yi[:] = yi_new[:]
        
        #add the step to the arrays-1,:
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new [0:len(x)-1,:] = y
        y_new [-1,:] = yi[:]
        del y
        y = y_new
        
        #prevent too many iterations
        if(i>imax):
            
            print("Maximum iteration reached.")
            raise StopIteration("Iteration number = ", i)
            
        #iterate
        i +=1
        
        #output some information
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i,xi, h_step, b)
        print(s)
        
        #break if new xi is == b
        if(xi==b):
            flag = 0
            
    return x,y


# Perform the integration

In [71]:
a = 0.0
b = 2.0 * np.pi

y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

tolerance = 1.0e-6

#perform the integration
x,y = cash_wrap(dydx,a,b,y_0,tolerance)

i =   1	x = 0.00004912	h = 0.00004912	b=6.28318531
i =   2	x = 0.00009953	h = 0.00005041	b=6.28318531
i =   3	x = 0.00015007	h = 0.00005054	b=6.28318531
i =   4	x = 0.00020063	h = 0.00005055	b=6.28318531
i =   5	x = 0.00025118	h = 0.00005056	b=6.28318531
i =   6	x = 0.00030174	h = 0.00005056	b=6.28318531
i =   7	x = 0.00035229	h = 0.00005056	b=6.28318531
i =   8	x = 0.00040285	h = 0.00005056	b=6.28318531
i =   9	x = 0.00045340	h = 0.00005056	b=6.28318531
i =  10	x = 0.00050396	h = 0.00005056	b=6.28318531
i =  11	x = 0.00055451	h = 0.00005056	b=6.28318531
i =  12	x = 0.00060507	h = 0.00005056	b=6.28318531
i =  13	x = 0.00065563	h = 0.00005056	b=6.28318531
i =  14	x = 0.00070618	h = 0.00005056	b=6.28318531
i =  15	x = 0.00075674	h = 0.00005056	b=6.28318531
i =  16	x = 0.00080729	h = 0.00005056	b=6.28318531
i =  17	x = 0.00085785	h = 0.00005056	b=6.28318531
i =  18	x = 0.00090840	h = 0.00005056	b=6.28318531
i =  19	x = 0.00095896	h = 0.00005056	b=6.28318531
i =  20	x = 0.00100951	h = 0.00

i = 894	x = 0.04520287	h = 0.00005058	b=6.28318531
i = 895	x = 0.04525345	h = 0.00005058	b=6.28318531
i = 896	x = 0.04530404	h = 0.00005058	b=6.28318531
i = 897	x = 0.04535462	h = 0.00005058	b=6.28318531
i = 898	x = 0.04540520	h = 0.00005058	b=6.28318531
i = 899	x = 0.04545578	h = 0.00005058	b=6.28318531
i = 900	x = 0.04550636	h = 0.00005058	b=6.28318531
i = 901	x = 0.04555695	h = 0.00005058	b=6.28318531
i = 902	x = 0.04560753	h = 0.00005058	b=6.28318531
i = 903	x = 0.04565811	h = 0.00005058	b=6.28318531
i = 904	x = 0.04570869	h = 0.00005058	b=6.28318531
i = 905	x = 0.04575928	h = 0.00005058	b=6.28318531
i = 906	x = 0.04580986	h = 0.00005058	b=6.28318531
i = 907	x = 0.04586044	h = 0.00005058	b=6.28318531
i = 908	x = 0.04591102	h = 0.00005058	b=6.28318531
i = 909	x = 0.04596161	h = 0.00005058	b=6.28318531
i = 910	x = 0.04601219	h = 0.00005058	b=6.28318531
i = 911	x = 0.04606277	h = 0.00005058	b=6.28318531
i = 912	x = 0.04611335	h = 0.00005058	b=6.28318531
i = 913	x = 0.04616394	h = 0.00

i = 1491	x = 0.07541291	h = 0.00005063	b=6.28318531
i = 1492	x = 0.07546354	h = 0.00005063	b=6.28318531
i = 1493	x = 0.07551417	h = 0.00005063	b=6.28318531
i = 1494	x = 0.07556480	h = 0.00005063	b=6.28318531
i = 1495	x = 0.07561543	h = 0.00005063	b=6.28318531
i = 1496	x = 0.07566605	h = 0.00005063	b=6.28318531
i = 1497	x = 0.07571668	h = 0.00005063	b=6.28318531
i = 1498	x = 0.07576731	h = 0.00005063	b=6.28318531
i = 1499	x = 0.07581794	h = 0.00005063	b=6.28318531
i = 1500	x = 0.07586857	h = 0.00005063	b=6.28318531
i = 1501	x = 0.07591920	h = 0.00005063	b=6.28318531
i = 1502	x = 0.07596983	h = 0.00005063	b=6.28318531
i = 1503	x = 0.07602046	h = 0.00005063	b=6.28318531
i = 1504	x = 0.07607109	h = 0.00005063	b=6.28318531
i = 1505	x = 0.07612172	h = 0.00005063	b=6.28318531
i = 1506	x = 0.07617235	h = 0.00005063	b=6.28318531
i = 1507	x = 0.07622298	h = 0.00005063	b=6.28318531
i = 1508	x = 0.07627361	h = 0.00005063	b=6.28318531
i = 1509	x = 0.07632424	h = 0.00005063	b=6.28318531
i = 1510	x =

i = 2139	x = 0.10824364	h = 0.00005071	b=6.28318531
i = 2140	x = 0.10829434	h = 0.00005071	b=6.28318531
i = 2141	x = 0.10834505	h = 0.00005071	b=6.28318531
i = 2142	x = 0.10839575	h = 0.00005071	b=6.28318531
i = 2143	x = 0.10844646	h = 0.00005071	b=6.28318531
i = 2144	x = 0.10849717	h = 0.00005071	b=6.28318531
i = 2145	x = 0.10854787	h = 0.00005071	b=6.28318531
i = 2146	x = 0.10859858	h = 0.00005071	b=6.28318531
i = 2147	x = 0.10864929	h = 0.00005071	b=6.28318531
i = 2148	x = 0.10869999	h = 0.00005071	b=6.28318531
i = 2149	x = 0.10875070	h = 0.00005071	b=6.28318531
i = 2150	x = 0.10880141	h = 0.00005071	b=6.28318531
i = 2151	x = 0.10885211	h = 0.00005071	b=6.28318531
i = 2152	x = 0.10890282	h = 0.00005071	b=6.28318531
i = 2153	x = 0.10895353	h = 0.00005071	b=6.28318531
i = 2154	x = 0.10900424	h = 0.00005071	b=6.28318531
i = 2155	x = 0.10905494	h = 0.00005071	b=6.28318531
i = 2156	x = 0.10910565	h = 0.00005071	b=6.28318531
i = 2157	x = 0.10915636	h = 0.00005071	b=6.28318531
i = 2158	x =

i = 2700	x = 0.13671369	h = 0.00005080	b=6.28318531
i = 2701	x = 0.13676449	h = 0.00005080	b=6.28318531
i = 2702	x = 0.13681528	h = 0.00005080	b=6.28318531
i = 2703	x = 0.13686608	h = 0.00005080	b=6.28318531
i = 2704	x = 0.13691687	h = 0.00005080	b=6.28318531
i = 2705	x = 0.13696767	h = 0.00005080	b=6.28318531
i = 2706	x = 0.13701847	h = 0.00005080	b=6.28318531
i = 2707	x = 0.13706926	h = 0.00005080	b=6.28318531
i = 2708	x = 0.13712006	h = 0.00005080	b=6.28318531
i = 2709	x = 0.13717086	h = 0.00005080	b=6.28318531
i = 2710	x = 0.13722165	h = 0.00005080	b=6.28318531
i = 2711	x = 0.13727245	h = 0.00005080	b=6.28318531
i = 2712	x = 0.13732325	h = 0.00005080	b=6.28318531
i = 2713	x = 0.13737405	h = 0.00005080	b=6.28318531
i = 2714	x = 0.13742484	h = 0.00005080	b=6.28318531
i = 2715	x = 0.13747564	h = 0.00005080	b=6.28318531
i = 2716	x = 0.13752644	h = 0.00005080	b=6.28318531
i = 2717	x = 0.13757724	h = 0.00005080	b=6.28318531
i = 2718	x = 0.13762804	h = 0.00005080	b=6.28318531
i = 2719	x =

...

i = 7852	x = 0.40242892	h = 0.00005270	b=6.28318531
i = 7853	x = 0.40248162	h = 0.00005270	b=6.28318531
i = 7854	x = 0.40253432	h = 0.00005270	b=6.28318531
i = 7855	x = 0.40258702	h = 0.00005270	b=6.28318531
i = 7856	x = 0.40263972	h = 0.00005270	b=6.28318531
i = 7857	x = 0.40269242	h = 0.00005270	b=6.28318531
i = 7858	x = 0.40274512	h = 0.00005270	b=6.28318531
i = 7859	x = 0.40279782	h = 0.00005270	b=6.28318531
i = 7860	x = 0.40285052	h = 0.00005270	b=6.28318531
i = 7861	x = 0.40290323	h = 0.00005270	b=6.28318531
i = 7862	x = 0.40295593	h = 0.00005270	b=6.28318531
i = 7863	x = 0.40300863	h = 0.00005270	b=6.28318531
i = 7864	x = 0.40306134	h = 0.00005270	b=6.28318531
i = 7865	x = 0.40311404	h = 0.00005270	b=6.28318531
i = 7866	x = 0.40316675	h = 0.00005271	b=6.28318531
i = 7867	x = 0.40321946	h = 0.00005271	b=6.28318531
i = 7868	x = 0.40327216	h = 0.00005271	b=6.28318531
i = 7869	x = 0.40332487	h = 0.00005271	b=6.28318531
i = 7870	x = 0.40337758	h = 0.00005271	b=6.28318531
i = 7871	x =

i = 8409	x = 0.43187437	h = 0.00005304	b=6.28318531
i = 8410	x = 0.43192740	h = 0.00005304	b=6.28318531
i = 8411	x = 0.43198044	h = 0.00005304	b=6.28318531
i = 8412	x = 0.43203348	h = 0.00005304	b=6.28318531
i = 8413	x = 0.43208652	h = 0.00005304	b=6.28318531
i = 8414	x = 0.43213956	h = 0.00005304	b=6.28318531
i = 8415	x = 0.43219260	h = 0.00005304	b=6.28318531
i = 8416	x = 0.43224564	h = 0.00005304	b=6.28318531
i = 8417	x = 0.43229868	h = 0.00005304	b=6.28318531
i = 8418	x = 0.43235172	h = 0.00005304	b=6.28318531
i = 8419	x = 0.43240476	h = 0.00005304	b=6.28318531
i = 8420	x = 0.43245780	h = 0.00005304	b=6.28318531
i = 8421	x = 0.43251085	h = 0.00005304	b=6.28318531
i = 8422	x = 0.43256389	h = 0.00005304	b=6.28318531
i = 8423	x = 0.43261693	h = 0.00005304	b=6.28318531
i = 8424	x = 0.43266998	h = 0.00005305	b=6.28318531
i = 8425	x = 0.43272302	h = 0.00005305	b=6.28318531
i = 8426	x = 0.43277607	h = 0.00005305	b=6.28318531
i = 8427	x = 0.43282912	h = 0.00005305	b=6.28318531
i = 8428	x =

i = 8965	x = 0.46146319	h = 0.00005340	b=6.28318531
i = 8966	x = 0.46151659	h = 0.00005340	b=6.28318531
i = 8967	x = 0.46157000	h = 0.00005340	b=6.28318531
i = 8968	x = 0.46162340	h = 0.00005341	b=6.28318531
i = 8969	x = 0.46167681	h = 0.00005341	b=6.28318531
i = 8970	x = 0.46173021	h = 0.00005341	b=6.28318531
i = 8971	x = 0.46178362	h = 0.00005341	b=6.28318531
i = 8972	x = 0.46183703	h = 0.00005341	b=6.28318531
i = 8973	x = 0.46189044	h = 0.00005341	b=6.28318531
i = 8974	x = 0.46194385	h = 0.00005341	b=6.28318531
i = 8975	x = 0.46199726	h = 0.00005341	b=6.28318531
i = 8976	x = 0.46205067	h = 0.00005341	b=6.28318531
i = 8977	x = 0.46210408	h = 0.00005341	b=6.28318531
i = 8978	x = 0.46215750	h = 0.00005341	b=6.28318531
i = 8979	x = 0.46221091	h = 0.00005341	b=6.28318531
i = 8980	x = 0.46226432	h = 0.00005341	b=6.28318531
i = 8981	x = 0.46231774	h = 0.00005341	b=6.28318531
i = 8982	x = 0.46237115	h = 0.00005342	b=6.28318531
i = 8983	x = 0.46242457	h = 0.00005342	b=6.28318531
i = 8984	x =

i = 9537	x = 0.49212638	h = 0.00005382	b=6.28318531
i = 9538	x = 0.49218020	h = 0.00005382	b=6.28318531
i = 9539	x = 0.49223402	h = 0.00005382	b=6.28318531
i = 9540	x = 0.49228783	h = 0.00005382	b=6.28318531
i = 9541	x = 0.49234165	h = 0.00005382	b=6.28318531
i = 9542	x = 0.49239547	h = 0.00005382	b=6.28318531
i = 9543	x = 0.49244929	h = 0.00005382	b=6.28318531
i = 9544	x = 0.49250311	h = 0.00005382	b=6.28318531
i = 9545	x = 0.49255694	h = 0.00005382	b=6.28318531
i = 9546	x = 0.49261076	h = 0.00005382	b=6.28318531
i = 9547	x = 0.49266458	h = 0.00005382	b=6.28318531
i = 9548	x = 0.49271840	h = 0.00005382	b=6.28318531
i = 9549	x = 0.49277223	h = 0.00005382	b=6.28318531
i = 9550	x = 0.49282605	h = 0.00005383	b=6.28318531
i = 9551	x = 0.49287988	h = 0.00005383	b=6.28318531
i = 9552	x = 0.49293371	h = 0.00005383	b=6.28318531
i = 9553	x = 0.49298753	h = 0.00005383	b=6.28318531
i = 9554	x = 0.49304136	h = 0.00005383	b=6.28318531
i = 9555	x = 0.49309519	h = 0.00005383	b=6.28318531
i = 9556	x =

StopIteration: ('Iteration number = ', 10001)

In [69]:
plt.plot(x,y[:,0],'o',label='y(x)')
plt.plot(x,y[:,1],'o',label='dydx(x)')
xx = np.linespace(0,2.0*np.pi,1000)
plt.plot(xx,np.sin(xx),label='sin(x)')
plt.plot(xx,np.cose(xx),label='cos(x)')
plt.xlabel('x')
plt.ylabel('y,dy/dx')
plt.legend(frameon_False)

NameError: name 'y' is not defined

# Plot the error

In [70]:
sine = np.sin(x)
cosine = np.cos(x)

y_error = (y[:,1]-sine)
dydx_error = (y[:,1]-cosine)

plt.plot(x,y_error, label="y(x) Error")
plt.plot(x, dydx_error, lebel="dydx(x) Error")
plt.legend(frameon=False)

NameError: name 'y' is not defined